<a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

# 部署

## “模型”由哪些文件构成？

我们把经过训练的网络从训练环境中移出, “部署” 至应用程序，使其发挥其作用。我们将在 DIGITS 中，从上次中断处开始操作。

DIGITS 会将需要部署的文件存放于可供下载或指向的目录中。由于我们将在训练模型的同一服务器上对其进行部署，因而可以指向 DIGITS 生成的文件夹路径。

<a href="/digits/">打开 DIGITS</a>

从 DIGITS 主页面，选择已命名为“Dogs vs. Cats”（狗和猫）的模型。

DIGITS 的模型“工作页面”是您在创建模型、训练模型及/或在 DIGITS“模型”选项卡下选择模型时看到的页面。工作目录位于左上方。

![](images/ModelJobView.PNG)

**复制工作目录（上文高亮部分），并替换以下代码块中的 ##FIXME##。完成替换之后，执行此单元 (Shift+Enter) 以将其存储到变量 <code>MODEL_JOB_DIR</code> 中**

In [ ]:
MODEL_JOB_DIR = '##FIXME##'  ## Remember to set this to be the job directory for your model
!ls $MODEL_JOB_DIR

复制并粘贴成功后，您将看到该目录中所有文件的列表。如果以下说明与您所见不符，请检查复制/粘贴操作是否成功。

我们的“模型”同样包含两个文件：网络结构和权重。

网络结构文件名为 ```deploy.prototxt```，权重则在最新生成的快照文件中（文件名为 ```snapshot_iter_#.caffemodel.```）在本例中，快照编号 735 包含 5 次训练全部完成后习得的权重。

In [ ]:
ARCHITECTURE = MODEL_JOB_DIR + '/' + 'deploy.prototxt'
WEIGHTS = MODEL_JOB_DIR + '/' + 'snapshot_iter_735.caffemodel'
print ("Filepath to Architecture = " + ARCHITECTURE)
print("Filepath to weights = "+ WEIGHTS)

接下来，我们需要确保正在构建的程序可以读取并处理这些文件。在部署中，我们需安装模型训练时所用的深度学习框架，以便能够解析它们。稍后，我们会在本课程中学习如何将模型部署至无需安装框架的环境中。我们还需使用 GPU的并行处理能力。我们的模型包含数十万次操作运算，可通过并行计算予以显著加速。

In [ ]:
import caffe
caffe.set_mode_gpu()

接下来，我们将创建名为“net”的“分类器”对象。工作流程越常见，您就越容易利用现有工具创建项目。在本例中，图像分类十分常见，因此下一个代码块只需使用网络结构文件和权重文件以及部分数据，简化了常见操作。

In [ ]:
# Initialize the Caffe model using the model trained in DIGITS
net = caffe.Classifier(ARCHITECTURE, WEIGHTS,  
                       channel_swap =(2, 1, 0), #Color images have three channels, Red, Green, and Blue.
                       raw_scale=255) #Each pixel value is a number between 0 and 255
                       #Each "channel" of our images are 256 x 256 

“分类器”这个类包括一种“预测”方法，该方法会提取上文所定义图像的输入，输出该图像属于每一类别的概率。


## 创建预期输入：预处理

我们首先来进行一项简单的任务：尝试从数据集中正确分类已标记图像。我们可以通过运行下方单元来加载图像并进行查看。

In [ ]:
import matplotlib.pyplot as plt #matplotlib.pyplot allows us to visualize results
input_image= caffe.io.load_image('/dli/data/dogscats/train/cats/cat.10941.jpg')
plt.imshow(input_image)
plt.show()

虽然我们已获得此图像，但它并非网络所期望的“输入”。

在准备推理数据时，我们需遵循一条黄金法则：<pre>训练前的所有操作均需在推理之前完成</pre>

在上一节中，您看到了 DIGITS 在训练模型时生成的文件。而在本节，我们将检测 DIGITS 在创建数据集时生成的文件。

当您从模型页面“狗和猫”中选择数据集及/或从 DIGITS“数据集”选项卡下选择数据集时，便可找到所训练**数据集**的工作目录。工作目录与模型所在位置相同，只不过编号应会有所不同。

![](images/datasetjobdir.PNG)

使用此工作目录替换 ##FIXME##，并执行下方代码以将 DATA_JOB_DIR 设置为正确的文件路径，然后检查此路径下的内容：

In [ ]:
DATA_JOB_DIR = '##FIXME##'  ## Remember to set this to be the job directory for your model
!ls $DATA_JOB_DIR

同样，此处会提供超过您目前所需的更多信息。您*能够*了解有关数据科学和数据准备的海量信息，并会在解决各类深度学习问题后获得更清晰的认知。在本例中，DIGITS 在训练之前执行了两个步骤。我们称之为*预处理*。

1) DIGITS 将图像调整为 256X256 大小的彩色图像  

In [ ]:
import cv2
input_image=cv2.resize(input_image, (256, 256), 0,0)
plt.imshow(input_image)
plt.show()

2) DIGITS 通过从每张图像中减除均值图像以将图像*标准化*，以此减少训练所需的计算量。

加载均值图像，并将其从以下测试图像中减除：

In [ ]:
mean_image = caffe.io.load_image(DATA_JOB_DIR+'/mean.jpg')
ready_image = input_image-mean_image

我们现已取得原始数据，并已将其转换为网络所期望的数据。接下来，我们来看一下网络所创建的输出。

## 前向传播：使用您的模型

这就是我们的关注所在。我们来看看这个函数：
<code>prediction = net.predict([grid_square])</code>.

与其他任一 [函数](https://www.khanacademy.org/computing/computer-programming/programming#functions) 类似，<code>net.predict</code> 也会传递输入 <code>ready_image</code>，并返回输出 <code>prediction</code>。但与其他函数不同，该函数并不遵循一组步骤，而是逐层开展矩阵数学运算，以将图像转换为概率向量。

运行下方单元，以查看对以上已标记数据作出的预测。

In [ ]:
# make prediction
prediction = net.predict([ready_image])
print prediction

结果很有趣，但并未包含太多信息。我们的网络获取到 256x256 大小的标准化彩色图像，并生成一个长度为 2 的向量。

## 生成有用的输出：后处理

此时，我们便可真正构建想要的任何内容。而唯一的局限在于您的编程经验。在进行创意操作之前，我们先来构建一些基本内容。此代码将能确定网络输出“狗”的概率是否高于“猫”。若为此，当狗接近模拟的狗门时，其将显示适当的图像。否则，若网络确定门前是只猫，图像便会呈现我们希望看到的内容。

In [ ]:
print("Input image:")
plt.imshow(input_image)
plt.show()

print("Output:")
if prediction.argmax()==0:
    print "Sorry cat:( https://media.giphy.com/media/jb8aFEQk3tADS/giphy.gif"
else:
    print "Welcome dog! https://www.flickr.com/photos/aidras/5379402670"

此时，我们已在此处做好所有准备工作，您现在可以对狗门可能会看到的图像进行测试。

In [ ]:
##Create an input our network expects
input_image= caffe.io.load_image('/dli/data/fromnest.PNG')
input_image=cv2.resize(input_image, (256, 256), 0,0)
ready_image = input_image-mean_image
##Treat our network as a function that takes an input and generates an output
prediction = net.predict([ready_image])
print("Input Image:")
plt.imshow(input_image)
plt.show()
print(prediction)
##Create a useful output
print("Output:")
if prediction.argmax()==0:
    print "Sorry cat:( https://media.giphy.com/media/jb8aFEQk3tADS/giphy.gif"
else:
    print "Welcome dog! https://www.flickr.com/photos/aidras/5379402670"

基本上，我们已经为狗门挑战创建出模拟器。我们已创建一款应用程序，其能从摄像机中提取输入、将其转换为网络所期望的数据类型、生成输出，并将该输出转换为对用户有用的内容。

您能够了解如何通过正确的输出来轻松控制狗门的动力开关装置。在深度学习方面，您已掌握所需的一切！如要查看可在以上代码块中试用的其他图像<code>列表</code>，您可通过运行下方命令来找到测试图像（不用于训练的图像）。其中一些图像预计会输出错误分类。您需要不断测试这些图像直到取得满意的结果为止，然后再继续课程学习，以了解如何提高性能！

In [ ]:
!ls /dli/data/dogscats/test

## 融会贯通

我们来系统全面地理解一下该部署过程，看看它在此 Jupyter notebook 之外会是什么样。在 [pythondeployment.py](../../../../edit/tasks/task3/task/pythondeployment.py) 的 Python 文件中，您将看到与上文相同的代码，不过此处代码已合并至同一个文件。在课程末尾的“评估”一节中，您将使用此方法，所以现在请作一些了解。将文件路径插入到此处的测试图像，以对其进行可视化。 

In [ ]:
TEST_IMAGE = '/dli/data/dogscats/test/1.jpg'
display= caffe.io.load_image(TEST_IMAGE)
plt.imshow(display)
plt.show()

然后，运行我们的小型 python 应用程序，并按下方输入该图像。请忽略输出的大部分内容，并滚动至底部。（即使出现错误和警告也没关系。）

In [ ]:
!python pythondeployment.py $TEST_IMAGE 2>/dev/null

<a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"> <img src="images/DLI Header.png" alt="标题" style="width: 400px;"/> </a>